In [3]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

from random import seed
from random import randint
from sklearn.cluster import KMeans

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('../retail_data/data.csv', encoding = "ISO-8859-1")

In [5]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [11]:
df[df['StockCode']=='85123A']['Description'].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2302
CREAM HANGING HEART T-LIGHT HOLDER       9
?                                        1
wrongly marked carton 22804              1
Name: Description, dtype: int64

In [14]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [13]:
df[df['CustomerID'].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom


In [7]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [48]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['monetary_value'] = df['Quantity'] * df['UnitPrice']

df.rename(columns = {'Quantity':'frequency'}, inplace = True)

Just by looking at this dataframe I'm thinking of different tables. 
- Product table: product_id, stock_code, description, UnitPrice

- Invoice table: invoice_id, InvoiceNo

- Fact table: invoice_id, product_id, customer_id, country_id, date_id, Quantity

- time table: date_id, invoice_date, minutes, hours, day, week, month, quarter, year, dayofweek.

- Location: country_id, country

- Customers: customer_id


I'll do a little deep dive on the customers to know how many things we can extract from them just as if we were a real company

### Exploring customers with RFM

In [49]:
customer_panel = df.groupby('CustomerID').agg({'InvoiceDate': max,
                                               'monetary_value': sum,
                                               'frequency': sum}).reset_index()

customer_panel['recency'] = ((customer_panel['InvoiceDate'].max() - customer_panel['InvoiceDate'])
                             / np.timedelta64(1, 'D')).round(0)

customer_panel.drop(columns = ['InvoiceDate'], inplace = True)

In [50]:
def order_cluster(cluster_field_name, target_field_name, df, ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [51]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(customer_panel[['recency']])
customer_panel['RecencyCluster'] = kmeans.predict(customer_panel[['recency']])

In [52]:
customer_panel = order_cluster('RecencyCluster', 'recency', customer_panel, False)

In [53]:
customer_panel.groupby('RecencyCluster')['recency'].mean()

RecencyCluster
0    309.381423
1    190.251603
2     79.707834
3     17.916551
Name: recency, dtype: float64

In [54]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(customer_panel[['frequency']])
customer_panel['FrequencyCluster'] = kmeans.predict(customer_panel[['frequency']])

In [55]:
customer_panel = order_cluster('FrequencyCluster', 'frequency', customer_panel, True)

In [58]:
customer_panel.groupby('FrequencyCluster')['frequency'].mean()

FrequencyCluster
0       817.166513
1     22696.576923
2     64091.555556
3    196719.000000
Name: frequency, dtype: float64

In [59]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(customer_panel[['monetary_value']])
customer_panel['MonetaryCluster'] = kmeans.predict(customer_panel[['monetary_value']])

In [60]:
customer_panel = order_cluster('MonetaryCluster', 'monetary_value', customer_panel, True)

In [61]:
customer_panel.groupby('MonetaryCluster')['monetary_value'].mean()

MonetaryCluster
0      1150.834871
1     14989.430833
2     71423.516000
3    241136.560000
Name: monetary_value, dtype: float64

In [65]:
customer_panel['RFMScore'] = customer_panel['RecencyCluster'] + customer_panel['FrequencyCluster'] + customer_panel['MonetaryCluster']

In [77]:
customer_panel.groupby('RFMScore').agg({'recency':np.mean,
                                        'frequency':np.mean,
                                        'monetary_value':np.mean,
                                        'CustomerID':'count'})

,recency,frequency,monetary_value,CustomerID
RFMScore,,,,
0,309.381423,192.144269,359.771245,506
1,190.180064,323.369775,539.891063,622
2,79.946396,606.779113,967.916499,1082
3,18.663400,1006.726605,1663.241843,2041
4,8.494253,6476.195402,12437.214368,87
5,6.312500,21680.437500,28965.437500,16
6,8.125000,26675.500000,54267.191250,8
7,6.571429,63381.857143,91030.744286,7
8,4.000000,66575.500000,221960.330000,2
